In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/cev/dl/log-analytics


In [2]:
import torch
import numpy as np
import pandas as np
import pickle

In [3]:
deck1 = torch.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr2e-05-dsver5-os40_2-deck11.pth")
deck2 = torch.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr2e-05-dsver5-os40_2-deck2.pth")

In [4]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([152791, 768]), torch.Size([1418916, 768]))

In [5]:
deck1["feat_"] = deck1["feat"].cuda()
deck2["feat_"] = deck2["feat"].cuda()

In [6]:
j = 0
shown = False

In [7]:
dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
values_, indices_ = dist_.topk(4, largest=False)
tlevels = deck1["tlevel"][indices_]
print("values:", values_)
print("indices:", indices_)
print("id:", deck1["tlevel"][indices_])
j += 1

values: tensor([0.0015, 0.0027, 0.0027, 0.0028], device='cuda:0')
indices: tensor([ 6308, 43056,  6171, 15030], device='cuda:0')
id: tensor([0, 0, 0, 0])


In [34]:
# while shown or (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5] and values_[0] < 0.2):
while shown or (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5]):
    shown = False

    dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
    values_, indices_ = dist_.topk(4, largest=False)
    tlevels = deck1["tlevel"][indices_]
    # print("values:", values)
    # print("indices:", indices)
    # print("id:", deck1["tlevel"][indices])
    j += 1

In [35]:
print("*  Submit :", deck2["otext"][j - 1])
print("----------------------------------------------------------------------------------------")
print("*   index :", j - 1)
print("*    dist :", " ".join(map(lambda x: f"{x:.4f}", values_.cpu().tolist())))
print("* fc-level:", deck2["fcfeat"][j - 1].argmax())
print("----------------------------------------------------------------------------------------")

for idx in indices_:
    print("* Level", deck1["tlevel"][idx].item(), ":", deck1["otext"][idx])
    print("----------------------------------------------------------------------------------------")

shown = True

*  Submit : month day time localhost sshd pid : Did not receive identification string from ip port
----------------------------------------------------------------------------------------
*   index : 64331
*    dist : 0.0042 0.1157 0.2792 0.3666
* fc-level: tensor(4)
----------------------------------------------------------------------------------------
* Level 4 : month day time localhost sshd pid : Did not receive identification string from ip port
----------------------------------------------------------------------------------------
* Level 4 : month 3 time localhost sshd pid : Did not receive identification string from ip port
----------------------------------------------------------------------------------------
* Level 4 : month day time sv260 sshd pid : Did not receive identification string from ip
----------------------------------------------------------------------------------------
* Level 4 : month 8 time sv260 sshd pid : Did not receive identification string from ip
--

---

4개가 다 일치할 때는 level별로 threshold를 따로 두는 방식으로 하고
* level0 - 좀 낮게 (비교군이 많기 때문에) - 0.3 ? 평균 dist가 0.3?
* level1 - 길이는 길지만 거의 비슷한 값들인듯? 대부분 0.01을 넘지 않음 - 0.2 or 0.1?
* level2 - 
* level3 - 꽤 높게 (변화무쌍하니까) - 1.0?
* level4 - 
* level5 - 꽤 높게 (변화무쌍하니까) - 1.0
* level6 - 나름 변화가 큼 - 0.8
* 섞인 경우 - 비슷한 케이스가 별로 없었다는 뜻이니까 낮게 - 0.2

2, 4, 6에 대해서는 아직 데이터가 더 모여야 하겠다

level0 길이가 긴거는 맞아도 dist가 높게(0.3이상) 나오고, 길이가 짧은건 틀려도 dist가 탖게 나오는 것 같은데...  
길이도 입력 인수로 해야할까?

※ TODO: 전체에 대해 dist 구한 다음에 각 level/군집별로 top dist 분포가 어떻게 나타나는지 그래프를 그려보자